#  BKLM Data Taking - Short Guide

you can find the original Page over [here](https://confluence.desy.de/display/BI/BKLM+Data+Taking+-+Short+Guide).


![VNC Setup](img/kek_vnc.png)


For the time beeing the Controlling of the HV is ignored. 

The setup uses the following Computers/shells.

|Shell |      Computer  |  Use |
|----------|-----------|------|
|KLM01_A   | klm01       | Start/Kill run |
|KLM01_B   | klm01       |   Auxiliary terminal to debug COPPERs |
|ttd11_A   | ttd11       |    Program FEEs; Include / mask sectors; Start reset triggers  |
|IDLAB_A   | idlab       | configure BF2   |
|ttd11_B   | ttd11       | Local run status |
|ttd11_C   | ttd11       | Global run status| 
|KLM01_C   | klm01       | Check COPPERS status|
|KLM01_D   | klm01       | Launch SLC     | 




In [1]:
import  config.config_kek as  config
from Linux_Helpers.remoteshell2 import remoteShell2
from Linux_Helpers.remoteshell3 import remoteShell3

#from equipment.ScintilatorReadout import ScintilatorReadout
from equipment.py_ftsw import py_ftsw
from script.prog_fee import prog_fee
from script.prepft import prepft
import time

In [2]:
from config.load_known_hosts import load_known_hosts
from Linux_Helpers.ssh_host import shell_factory


In [3]:
from config.RCL_Load_Config import RCL_Load_Config


In [4]:
from config.ftsw_load_config import ftsw_load_config
from equipment.py_ftsw import create_ftsw


In [5]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

from ipywidgets import widgets
import ipython_blocking
ipython_blocking.load_ipython_extensions()

from IPython.display import clear_output




## Setting up the envirement

Since we want to use remote terminals, we have to set them up. Since this notebook is not (yet) connected to the KEK setup all the shells are just dummies and most likley they will produce some errors.

In [36]:
file_name="config/known_host_KEK.xlsx"
KLM_Conf = "BFR_a_Not_working"

Known_hosts = load_known_hosts(file_name)
Sh_Factory = shell_factory(Known_hosts)
ftsw_conf = ftsw_load_config(file_name,KLM_Conf)

In [37]:
ftsw = create_ftsw(Sh_Factory ,ftsw_conf )

KLM01_A = Sh_Factory.get_shell('klm01')


class klm01:
    HostName = "127.0.0.1"
    UserName = "b2klm"
    IdentityFile = "config/b2klm"
    Port = 10040
    startCommand = ""
    endstring = "b2klm.klm@klm01"
    
    
    
KLM01_B = remoteShell3(host=klm01)

IDLAB_A = Sh_Factory.get_shell('idlab')


In [38]:
print(KLM01_B.sendAndRecieveRaw('ls',removePromt=False))



20150512.0507.dat             KEKFW/               run/
%backup%~*                    klm02.conf           run_ctrl7_OCT3.dat
basf2@                        klm02_weKLM.conf     scripts/
basf2_local/                  klm02_weKLM.conf~    setup.sh*
batchTesths*                  klm03.conf           SOCK_KLM01_klm01.txt
Belle2FileCatalog.xml         #klm.conf#           src/
bin/                          klm.conf             t3.txt
bklmEffi.root                 klm.conf~            temp/
daq_slc@                      klmslscan/           temp1.conf
daq_slc.copper@               log/                 temp2.conf
daq_slc-sl5/                  log180215-15:45      temp.conf
daq_slc-sl6/                  log180216-09:00      temp.conf~
diagnos_output.root           nsmd2.1970*          test2.txt
eb/                           o.txt                tmp/
epics/                        Output.sroot         tmp3.sroot
externals/                    rcl_rev2_10.zip*     trg_scan_setgain_dbg.txt
fefw4/

In [ ]:
def f(DebugPrint):
    ftsw.debug(DebugPrint)
    
x=interact(f,DebugPrint=False)

In [ ]:
# don't go beyond here with Run All
assert False

Once the Notebook is connected to KEK, 
all of these commands should run without error.
If the command `remoteShell2` produces and error make sure the SSH_Tunnel is setup correctly and to PocketDAQ / Copper is running

# Preliminary steps


## Program the FEEs

- Where? `ttd11_A` (bottom-left terminal, left tab)
- How? launch the following commands:
```
./progfee.sh
./progfee_BB.sh
```


To try new firmware
- upload your bitfile into the `Firmware/` directory 
- modify the variable `latestFirmware` in the cell below
- execute the cell below

In [ ]:
latestFirmware = "181017_klmscint_simple_top.bit"
FW_source = "Firmware/" + latestFirmware
FW_target = "lastfw/" + latestFirmware
ftsw.shell.copyTo(FW_source, FW_target)

In [ ]:
from config.Dev_details_load import Dev_details_load
BB = Dev_details_load(file_name,KLM_Conf,'BB')
BF = Dev_details_load(file_name,KLM_Conf,'BB')

In [ ]:
BB.scrod_firmware = FW_target
BF.scrod_firmware = FW_target
print(ftsw.shell.sendAndRecieve("ls " + FW_target))

In [ ]:
ftsw.shell.sendAndRecieve("jtagft -191 -p1 chain")

In [ ]:
print(prog_fee(ftsw,BB))

In [ ]:
print(prog_fee(ftsw,BF))

possible errors 

```
too many devices found in the jtag chain (probably noise).
```


- Check: the last messages shown in the terminal are:

   `ALL KLM FEE PROG DONE!` 
   
   and 
   
   `ALL KLM BB FEE PROG DONE!` 
   
   In the meanwhile, the bottom-center terminal, left tab (`ttd11_B`) should be in an ERROR  state and become purple: it's fine.
   
In a future modification to this Notebook the status of `ttd11_B` should actually be checked.


## Program the BF2 sector

(Note added on 2018-05-01: BF2 is currently masked, so the commands this section can be skipped- Now BB2 is acting as BF2 for debug interface, but script names are the same for ease of use)


- Where? `IDLAB_A` (bottom-left terminal, right tab)
- How? launch the following commands:

```
  impact -batch imp_prog_BF2scrods.cmd
  impact -batch imp_prog_BF2dc.cmd
```




To change the version of the firmware that is being programmed to SCROD FPGAs:
- move your bitfile to the `Firmware` directory on this virtual machine
- specify the name of your bitfile under `latestFirmware` in the next cell
- execute the next four cells



In [ ]:
latestFirmware = "181017_klmscint_simple_top.bit"
FW_path = "/home/idlab/lastfw/"
BFIDLAB = FW_path+latestFirmware
IDLAB_A.copyTo("Firmware/"+latestFirmware, BFIDLAB)

In [ ]:
impactCommand = "imp_prog_BF2scrods_latest.cmd"

In [ ]:
def CreateImpactCommandFile (bitFile, cmdFile):
    # Batch command for programming SCRODs via command line
    outCmdFile = 'setMode -bs \nsetMode -bs \nsetMode -bs \nsetMode -bs \nsetCable -target "digilent_plugin DEVICE=SN:210249992992 FREQUENCY=10000000" \nIdentify -inferir  \nidentifyMPM  \nassignFile -p 1 -file "' + bitFile + '"\nProgram -p 1  \nsetCable -target "digilent_plugin DEVICE=SN:210249993114 FREQUENCY=10000000" \nIdentify -inferir  \nidentifyMPM  \nassignFile -p 1 -file "' + bitFile + '"\nProgram -p 1  \nsetMode -bs \nsetMode -bs \ndeleteDevice -position 1 \nsetMode -bs \nsetMode -ss \nsetMode -sm \nsetMode -hw140 \nsetMode -spi \nsetMode -acecf \nsetMode -acempm \nsetMode -pff'
    print(outCmdFile)
    with open(cmdFile, 'w') as f:
        f.write(outCmdFile)

In [ ]:
CreateImpactCommandFile(BFIDLAB, impactCommand)
IDLAB_A.copyTo(impactCommand, impactCommand)


In [ ]:
print(IDLAB_A.sendAndRecieve("source bashrc-ise.sh;impact -batch " + impactCommand))

In [ ]:

print(IDLAB_A.sendAndRecieve("source bashrc-ise.sh;impact -batch imp_prog_BF2dc.cmd"))

- Check: one of the last message shown in the terminal is:

```
INFO:iMPACT - '1': Checking done pin....done.   / /   '1': Programmed successfully.   / /   Elapsed time =  X sec.
```


  Ignore further error messages!  
  
  
  ```
  INFO:iMPACT - '1': Checking done pin....done.
'1': Programmed successfully.
Elapsed time =     10 sec.
ERROR:iMPACT - SSerial mode is no longer supported by impact software , please
   use 11.4 or older version of software.
```



## Prepare ttd11
- Where? `ttd11_A` (bottom-left terminal, left tab)
- How? launch the following commands:
```        
  resetft -191
```
  With EKLM: 
```  
cd withEKLM; ./prepft_butBF2_withEKLM.sh
```

   Without EKLM: 
```
./prepft_butBF2.sh
```




In [ ]:
useEKLM = False
if useEKLM:
    print(prepft(ftsw,config.klm_prepft_with_EKLM))
else:
    print(prepft(ftsw,BB))


with EKLM not yet tested

```python
useEKLM = False
if useEKLM:
    print(ttd11_A.sendAndRecieve("cd withEKLM; ./prepft_butBF2_withEKLM.sh"))
else:
    print(ttd11_A.sendAndRecieve("./prepft_butBF2.sh"))
```



- Check: 
  
  the last message shown in the terminal is:
```
resetting trigger
```

## Clear "b2linkdown" after reprograming

Usually you will see `b2linkdown` after reprograming, `b2linkdown` shown in the the bottom-center terminal. Run the following:


- At `KLM01_B` run:
```
~/run/scripts/batchTesths  
```

  `batchTesths` does "tesths -abcd" for all four coppers)


- At `ttd11_A` run:
```
./prepft_butBF2.sh
```

In [ ]:
print(Sh_Factory.run_on('cpr7001',"tesths -abcd"))

In [ ]:
print(Sh_Factory.run_on('cpr7002',"tesths -abcd"))

In [ ]:
print(Sh_Factory.run_on('cpr7003',"tesths -abcd"))

In [ ]:
print(Sh_Factory.run_on('cpr7004',"tesths -abcd"))

In [ ]:
print(prepft(ftsw,config.klm_prepft))

In [ ]:
print(ftsw.status())

then you should able to see `b2linkdown` was cleared, and `FTSW` in yellow READY state.

# Launch the Slow Control

- Where? `KLM01_D` (bottom-right terminal, right tab)
- How? launch the following commands:


```
    ~/run/scripts/load_brcl.sh rcl_rev2_40_th-100  
```

    
    (NOTE: Every time you send this command, you need to resend the following commands)

    NOTE: kill all that `watch` commands which are reading status of registers to avoid any conflict when you issue the load_brcl.sh. Kill the 
  
  ```
    watch ./checkpcr 
```
  
    and 
  
  ```
    watch ./dcregs.sh
```

    Don't keep `watch ./dcregs.sh` running! Unless after reprograming you want to check the status, you run

```
    watch ./dcreg.sh
```

    then kill it right away otherwise you will forget later.  

In [ ]:
from script.load_brcl import load_brcl
from script.load_klm_calset import load_klm_calset
RCL = RCL_Load_Config(file_name,KLM_Conf)

In [ ]:
load_brcl(shell_factory=Sh_Factory,RCL=RCL,Parallel=True)

- Check: the last message shown in the terminal is:

`window fine stop   / /   reg0033 = XXXXXXXX`

`Successfull configured ...`


- Then run this command to send MPPC gain settings to Layer 1 on Barrel - wait for all windows to finish completely before launching next one
```
cd ~/run/scripts
./load_klm_calset_L1.sh calset_171118_031700 200
```

In [ ]:
load_klm_calset(Sh_Factory,RCL,True)

- Wait for command to finish (it opens 16 concurrent terminals, so need to have X tunneling)

- after this run the following command:

    `./load_klm_calset_L2.sh calset_171118_060700 200`

- wait for previous command to completely finish, then set lookback and width for scintilaltor 

- make sure you do not use any other scint. lookback commands after this:
- `./set_addl.sh` (this command is very quiet) (if you see "Device or resource busy" message, just run this command again.)


In [ ]:
print(Sh_Factory.run_on('klm01',"cd ~/run/scripts; ./set_addl.sh"))

## Reset the trigger
- Where? `ttd11_A` (bottom-left terminal, left tab)
- How? launch the following command:

```
resetft -191
```


In [ ]:
ftsw.reset()

- Check: the bottom-center terminal, left tab `ttd11_B` should pass in a READY state and become yellow 

# Start a run


## Prepare basf2:
- Where? `@klm01` (top-left terminal, left tab)
- How? launch the following command:
  - With EKLM: 
  ```bash
  cd ~/basf2/release/daq/copper/daq_scripts/beklm; ./run_start_mono_klm_butBF2.sh
  ```
  - Without EKLM:
  ```bash
  ./run_start_mono_Barrel_butBF2.sh
  ```
- Check: eight little windows should appear in the display, the top four windows should all show "ffffffff" at tail.
- If you re-login klm01 as b2klm, need to setup basf2 environment, do the following (you can type "basf" then tab to see whether basf2 command is available, if it is not, it means basf2 environment is not set)
```bash
source ~b2klm/basf2/tools/setup_belle2
cd ~b2klm/basf2/release/; setuprel
cd ~/basf2/release/daq/copper/daq_scripts
``` 


### Setup basf2 environment and release
- typical output:
```
'cd /home/group/b2klm/basf2_local/release/daq/copper/daq_scripts\r'
```

In [73]:
KLM01_B.sendAndRecieveRaw("cd /home/group/b2klm/basf2_local/tools_161125/; source setup_belle2")
KLM01_B.sendAndRecieveRaw("cd /home/group/b2klm/basf2_local/release/; setuprel")
KLM01_B.sendAndRecieveRaw("cd /home/group/b2klm/basf2_local/release/daq/copper/daq_scripts")

'cd /home/group/b2klm/basf2_local/release/daq/copper/daq_scripts\r'

### Designate COPPER nodes and readout PCs for data acquisition



In [68]:
des_ser_COPPER_main = "/home/group/b2klm/basf2_local/release/daq/ropc/des_ser_COPPER_main"
des_ser_ROPC_main   = "/home/group/b2klm/basf2_local/release/daq/ropc/des_ser_ROPC_main"

FINNESEbitflag  = ["14","15","15","15"]
COPPER_hostname = [0 for i in range(4)]
COPPER_nodeID   = [0 for i in range(4)]
eb0_port        = [0 for i in range(4)]
COPPER_shell    = [0 for i in range(4)]

UseNSM_bool       = str(0)
NSMnodeName_cpr   = str(1)
NSMnodeName_ropc  = 'hogehoge1000'

for i in range(4):
    COPPER_hostname[i] = 'cpr700' + str(i+1)
    COPPER_nodeID[i] = '11744051' + str(i+3)
    eb0_port[i] = '3400' + str(i+1)
    COPPER_shell[i] = Sh_Factory.get_shell(COPPER_hostname[i])
    COPPER_shell[i].sendAndRecieve(des_ser_COPPER_main +" "+ COPPER_hostname[i] +" "+ COPPER_nodeID[i] +" "+ FINNESEbitflag[i] +" "+ UseNSM_bool +" "+ NSMnodeName_cpr+ "; sleep 3000000;")
    KLM01_B.sendAndRecieveRaw(des_ser_ROPC_main +" "+ COPPER_hostname[i] +" "+ UseNSM_bool +" "+ eb0_port[i] +" "+ NSMnodeName_ropc+ "; sleep 3000000;")

#cpr7001 = Sh_Factory.get_shell('cpr7001')
#cpr7002 = Sh_Factory.get_shell('cpr7002')
#cpr7003 = Sh_Factory.get_shell('cpr7003')
#cpr7004 = Sh_Factory.get_shell('cpr7004')
# Usage : RecvSendCOPPER.py <COPPER hostname> <COPPER node ID> <bit flag of FINNESEs> <Use NSM(Network Shared Memory)? yes=1/no=0> <NSM nodename>'
# bit flag of FINESSE
# slot a : 1, slot b : 2, slot c : 4, slot d : 8
# e.g. slot abcd -> bitflag=15, slot bd -> bit flag=10
#cpr7001.sendAndRecieve(des_ser_COPPER_main + " cpr7001 117440513 14 0 1; sleep 3000000;")
#cpr7002.sendAndRecieve(des_ser_COPPER_main + " cpr7002 117440514 15 0 1; sleep 3000000;")
#cpr7003.sendAndRecieve(des_ser_COPPER_main + " cpr7003 117440515 15 0 1; sleep 3000000;")
#cpr7004.sendAndRecieve(des_ser_COPPER_main + " cpr7004 117440516 15 0 1; sleep 3000000;")
#
# basf2 program BEFORE eb0(event builder 0) on a readout PC
#
# Usage : RecvStream0.py <COPPER hostname> <Use NSM(Network Shared Memory)? yes=1/no=0> <port # of eb0> <NSM nodename>
#KLM01_B.sendAndRecieveRaw(des_ser_ROPC_main + " cpr7001 0 34001 hogehoge1000; sleep 3000000;")
#KLM01_B.sendAndRecieveRaw(des_ser_ROPC_main + " cpr7002 0 34002 hogehoge1000; sleep 3000000;")
#KLM01_B.sendAndRecieveRaw(des_ser_ROPC_main + " cpr7003 0 34003 hogehoge1000; sleep 3000000;")
#KLM01_B.sendAndRecieveRaw(des_ser_ROPC_main + " cpr7004 0 34004 hogehoge1000; sleep 3000000;")


In [111]:
#print(KLM01_B.sendAndRecieveRaw("cat /home/group/b2klm/basf2_local/release/daq/rawdata/src/DesSerPrePC.cc"))
#print(KLM01_B.sendAndRecieveRaw("cat /home/group/b2klm/basf2_local/release/daq/rawdata/src/DesSerCOPPER.cc"))
#print(KLM01_B.sendAndRecieveRaw("cat /home/group/b2klm/basf2_local/release/daq/ropc/des_ser_COPPER_main.cc"))
#print(KLM01_B.sendAndRecieveRaw("cat /home/group/b2klm/basf2_local/release/daq/ropc/des_ser_ROPC_main.cc"))
#print(KLM01_A.copyFrom("/home/group/b2klm/basf2_local/release/daq/copper/daq_scripts/RecvStream2_mono.sh","RecvStream2_mono.sh"))

## Start the run:
- Where? `@ttd11` (bottom-left terminal, left tab)

- How? launch the following command:
```
trigft -191 <trigger_type> <trigger_rate> [<number_of_events>]
```
    (Launch "trigft" without any arguments to know which arguments the "trigft" command can receive)
    
    
- Check: the `bottom-center terminal`, `left tab` (@ttd11) should pass in a RUNNING state and become `blue` (smile)



## Wait:
- Where? `@klm01` (bottom-right terminal, left tab)
- Check: here the events recorded by each COPPER are shown: within each COPPER the event number must be the same (plus or minus 20 at high rate) and tipically, if N is the event number: N(cpr7001) < N(cpr7002) < N(cpr7003) < N(cpr7004)

# Stop a run


##  Stop the trigger:
- Where? `@ttd11` (bottom-left terminal, left tab)

- How? launch the following commands:
```
resetft -191
```
- Check: the last message shown in the terminal is: `resetting trigger`
- Check: in the `bottom-right terminal`, `left tab` (@klm01) all the numbers go to 0; in the `bottom-center terminal`, `left tab` (@ttd11) many numbers go to 0 (the trigger is correctly reset)



##  Stop the run:
- Where? `@klm01` (top-left terminal, left tab)

- How? launch the following commands:
```
    CTRL+C
    ~/bin/kill_basf2_all.sh
```
- Check: all the small windows are closed, in the terminal is shown the list of all the COPPERs and the last message shown is `Killed`


## Rename the output `.sroot` file:
- Where? `@klm01` (top-left terminal, left tab)

- How? launch the following command:
```
mv root_output.sroot <new_file_name>
```
  - to check naming criteria used to rename the previous runs files use
```
ls | grep .sroot
```
